In [ ]:
# This notebooks looks at primary mirror motion as a function of elevation for different pressures
## The idea is to derive the maximum amount of pressure we can use under the mirror before we start to see slippage
## This uses daytime data taken on 2021-08-09 in the M1_mirror_testing-2021-08-09.ipynb notebook

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient, rendezvous_dataframes

In [ ]:
%matplotlib inline

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

Query for all the `endReadout` events on the timespan of the night, elevation, and pmd data

In [ ]:
date='20210809'
test='m1_pressure_reduction_closed_dome'
run='test15'

if run == 'initial': 
    # Test run with standard LUT
    t1 = Time("2021-08-09T16:00:10.425", format='isot', scale='tai')
    t2 = Time("2021-08-09T16:15:37.425", format='isot', scale='tai')
    scale_factor=1.0
    start_log_msg = '[2021-07-08_Repeat_Focus_Test_START]'
    finish_log_msg = '[2021-07-08_Repeat_Focus_Test_END]'
elif run == 'test1':
    # day of 2021-07-08 - Test run with scalefactor=1.01
    # timestamp printed was not in tai and so is adjusted here
    scale_factor=1.01
    t1 = Time("2021-08-09T16:33:57.669", format='isot', scale='tai')+TimeDelta(37, format='sec', scale='tai')
    t2 = Time("2021-08-09T16:40:03.985", format='isot', scale='tai')+TimeDelta(37, format='sec', scale='tai')
# elif run == '':
    # VERIFY THIS
    # # # day of 2021-07-08 - Test3 run with scalefactor = 1.02  - not sure this was great, so redid it
    # t1 = Time("2021-08-09T16:42:32.166", format='isot', scale='tai')
    # t2 = Time("2021-08-09T16:48:41.015", format='isot', scale='tai')
elif run == 'test3b':
    # Starting Test3b, LUT offset by -2343.5619165039084 and is now 114846.7377685547
    # start time is 2021-08-09T17:35:49.273
    scale_factor=1.02
    t1 = Time("2021-08-09T17:35:49.273", format='isot', scale='tai')
    t2 = Time("2021-08-09T17:41:58.836", format='isot', scale='tai')  
elif run == 'test4b':
    # Starting Test4b, LUT offset by -4685.356377441411 and is now 112471.19332275391. This is a scale_factor of 1.04
    # start time is 2021-08-09T17:45:58.520
    scale_factor=1.04
    t1 = Time("2021-08-09T17:45:58.520", format='isot', scale='tai')
    t2 = Time("2021-08-09T17:52:11.260", format='isot', scale='tai')
elif run == 'test5b':
    # Starting Test5b, LUT offset by -5826.18582214356 and is now 111164.53867187501. This is a scale_factor of 1.05
    # start time is 2021-08-09T17:54:39.997
    scale_factor=1.05
    t1 = Time("2021-08-09T17:54:39.997", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:00:52.720", format='isot', scale='tai')
elif run == 'test6':
    # Starting Test6, LUT offset by -7011.117491455085 and is now 109878.9251586914. This is a scale_factor of 1.06
    # start time is 2021-08-09T18:03:36.186
    scale_factor=1.06
    t1 = Time("2021-08-09T18:03:36.186", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:09:43.131", format='isot', scale='tai')
elif run == 'test7':
    # Starting Test7, LUT offset by -8194.807733642585 and is now 109115.13186035157. This is a scale_factor of 1.07
    # start time is 2021-08-09T18:12:34.226
    scale_factor=1.07
    t1 = Time("2021-08-09T18:12:34.226", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:18:42.275", format='isot', scale='tai')
elif run == 'test8':
    # Starting Test8, LUT offset by -9361.959641601572 and is now 107938.9322631836. This is a scale_factor of 1.08
    # start time is 2021-08-09T18:21:44.821
    scale_factor=1.08
    t1 = Time("2021-08-09T18:21:44.821", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:27:48.037", format='isot', scale='tai')
elif run == 'test9':
    # Starting Test9, LUT offset by -10496.981732299813 and is now 106282.99472656251. This is a scale_factor of 1.09
    # start time is 2021-08-09T18:29:40.394
    scale_factor=1.09
    t1 = Time("2021-08-09T18:29:40.394", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:35:45.856", format='isot', scale='tai')
elif run == 'test10':
    # Starting Test10, LUT offset by -11691.718571777354 and is now 105477.11915283203. This is a scale_factor of 1.1
    # start time is 2021-08-09T18:41:38.825
    scale_factor=1.10
    t1 = Time("2021-08-09T18:41:38.825", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:47:42.719", format='isot', scale='tai')
elif run == 'test11':
    # Starting Test11, LUT offset by 1166.5417149658213 and is now 118310.10903320313. This is a scale_factor of 0.99
    # start time is 2021-08-09T18:51:45.505
    scale_factor=0.99
    t1 = Time("2021-08-09T18:51:45.505", format='isot', scale='tai')
    t2 = Time("2021-08-09T18:57:53.278", format='isot', scale='tai')
elif run == 'test12':
    # Starting Test12, LUT offset by 2340.0690876464864 and is now 119574.68140869141. This is a scale_factor of 0.98
    # start time is 2021-08-09T19:02:46.869
    scale_factor=0.98
    t1 = Time("2021-08-09T19:02:46.869", format='isot', scale='tai')
    t2 = Time("2021-08-09T19:06:53.919", format='isot', scale='tai')
elif run == 'test13':
    # Starting Test13, LUT offset by 3515.3428747558623 and is now 120534.15728759766. This is a scale_factor of 0.97
    # start time is 2021-08-09T19:20:57.838
    scale_factor=0.97
    t1 = Time("2021-08-09T19:20:57.838", format='isot', scale='tai')
    t2 = Time("2021-08-09T19:25:04.316", format='isot', scale='tai')
elif run == 'test14':
    # Starting Test14, LUT offset by 4690.574579589847 and is now 121645.12935791016. This is a scale_factor of 0.96
    # start time is 2021-08-09T19:27:41.837
    scale_factor=0.96
    t1 = Time("2021-08-09T19:27:41.837", format='isot', scale='tai')
    t2 = Time("2021-08-09T19:31:44.407", format='isot', scale='tai')
elif run == 'test15':
    # Starting Test15, LUT offset by 5864.375487060553 and is now 122909.70173339844. This is a scale_factor of 0.95
    # start time is 2021-08-09T19:34:32.863
    scale_factor=0.95
    t1 = Time("2021-08-09T19:34:32.863", format='isot', scale='tai')
    t2 = Time("2021-08-09T19:38:35.766", format='isot', scale='tai') 
else:
    raise IOError('Not a valid input')

In [ ]:
# # # day of 2021-07-08 - Test3 run with scalefactor = 1.02  - not sure this was great, so redid it
# t1 = Time("2021-08-09T16:42:32.166", format='isot', scale='tai')
# t2 = Time("2021-08-09T16:48:41.015", format='isot', scale='tai')

In [ ]:
# # # day of 2021-07-08 - Test4 run with scalefactor = 1.04 - ignore, redoing
# t1 = Time("2021-08-09T16:50:24.872", format='isot', scale='tai')
# t2 = Time("2021-08-09T16:59:24.872", format='isot', scale='tai')

In [ ]:
# # day of 2021-07-08 - Test4 (didn't increment) run with scalefactor = 1.06 = BAD - Sensor dropped out
# t1 = Time("2021-08-09T16:59:24.872", format='isot', scale='tai')
# t2 = Time("2021-08-09T17:05:32.895", format='isot', scale='tai')

In [ ]:
base0 = await efd_client.select_time_series("lsst.sal.Script.logevent_logMessage", 
                                           ["message","level"], t1, t2)
base0

In [ ]:
base=base0

In [ ]:
el = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle1", ], t1, t2)
# mount reporting incorrect timestamp for the packed time series
el.index=el.index+pd.tseries.offsets.DateOffset(seconds=-37)

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position4", "position5"], t1, t2)

In [ ]:
pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure", ["pressure"], t1, t2)

In [ ]:
#base

In [ ]:
tmp = rendezvous_dataframes(base, el)

In [ ]:
tmp2= rendezvous_dataframes(tmp, pmd)
tmp3= rendezvous_dataframes(tmp2, pressure)
raw_data = tmp3

In [ ]:
position0_offset = (raw_data.position0[0])
position1_offset = (raw_data.position1[0])
position2_offset = (raw_data.position2[0])
position3_offset = (raw_data.position3[0])
position5_offset = (raw_data.position5[0]) # gauge got moved to position 5

In [ ]:
raw_data=raw_data.drop_duplicates(subset=['elevationCalculatedAngle1','message','pressure'])
print(raw_data[['elevationCalculatedAngle1','pressure']])

In [ ]:
# from scipy import linalg
arr_len = len(raw_data.position0)
coeff_arr = np.zeros((arr_len,3))
theta_arr = np.zeros((arr_len))
phi_arr = np.zeros((arr_len))
piston_arr = np.zeros((arr_len))
for i in np.arange(arr_len):
    # X, Y, Z
    set2=np.array((  41.0, 468.0, (raw_data.position2[i]-position2_offset)))
    set3=np.array(( 384.0,-269.0, (raw_data.position3[i]-position3_offset)))
    set4=np.array((-425.0,-198.0, (raw_data.position5[i]-position5_offset)))

    # Vector PQ crossed with Vector PR
    normal = np.cross(set3-set2,set4-set2) # gives a,b,c

    D= -normal[0]*set2[0] - normal[1]*set2[1] - normal[2]*set2[2]  # Constant in plane equation
    # equation 
    
    phi_from_normal = (np.pi/2+np.arctan2(normal[2], normal[1])) * 206265
    theta_from_normal = (np.pi/2+np.arctan2(normal[2], normal[0])) * 206265
    # find z at the origin to represent piston
    Z_origin = -D/normal[2]
    
    # Measure rotation about the Y-axis (perpendicular to elevation)
    # So this is TIP and should result in motion in azimuth
    # get slope by looking at Y=0, X=400
    x_pt=400; y_pt=0
    #Z_at_x_pt= C[0]*x_pt + C[1]*0.0 + C[2]
    Z_at_x_pt= (-D - normal[0]*x_pt - normal[1]*0.0)/normal[2]
    theta = np.arctan2(Z_at_x_pt-Z_origin, x_pt) * 206265 # arcsec
        
    # Measure rotation about the X-axis (aligned to elevation)
    # this is TILT and should result in motion in elevation
    # get slope by looking at Y=0, X=400    
    x_pt=0; y_pt=400
    Z_at_y_pt= (-D - normal[0]*x_pt - normal[1]*y_pt)/normal[2]
    phi = np.arctan2(Z_at_y_pt-Z_origin, y_pt) * 206265 # arcsec

    theta_arr[i] = theta # arcsec
    phi_arr[i] = phi    # arcsec
    piston_arr[i] = Z_origin
    
#     print(f'theta_from_normal is {theta_from_normal:0.2f}, phi_from_normal is {phi_from_normal:0.2f}')
#     print(f'theta_arr is {theta_arr[i]:0.2f} [arcsec], phi_arr is {phi_arr[i]:0.2f} [arcsec]')

In [ ]:
import copy
calc_data=copy.copy(raw_data)
calc_data['m1_tip']=theta_arr
calc_data['m1_tilt']=phi_arr
calc_data['m1_piston']=piston_arr

calc_data['m1_y_pos']=(raw_data.position0-position0_offset)*np.cos(10*np.pi/180)
calc_data['m1_x_pos']=(raw_data.position1-position1_offset)*np.cos(10*np.pi/180)

In [ ]:
#write data to disk
filename="data/"+date+'_'+test+"_"+run+"_metadata.csv"
calc_data.to_csv(filename)

In [ ]:
fig_height=5
fig_width=10
nwide=2; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (calc_data.m1_x_pos) # um
yvals = calc_data.elevationCalculatedAngle1

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals = (calc_data.m1_tip) # um
row1[1].plot(xvals, yvals, 'o-')
row1[1].set_xlabel('M1 tip [arcsec]')

xvals = (calc_data.m1_y_pos) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')

xvals = (calc_data.m1_tilt) # um
row2[1].plot(xvals, yvals, 'o-')
row2[1].set_xlabel('M1 tilt [arcsec]')

xvals = (calc_data.m1_piston) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals = (calc_data.pressure) # mm
row3[1].plot(xvals, yvals, 'o-')
row3[1].set_ylabel('Elevation [deg]')
row3[1].set_xlabel('Pressure [Pa]')

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     hex_elevation

# Now overplot the datasets

In [ ]:
import itertools
import matplotlib.cm as cm
from matplotlib import colors
runs = ['initial','test3b','test4b','test5b','test6','test7','test8','test9','test10','test11','test12','test13','test14','test15']
#sym= ['.-','x-','triangle_up-']

marker = itertools.cycle((',-', '+--', 'x-', 'v-', '*-')) 
#colors=['red','blue','green']
colors = cm.rainbow(np.linspace(0, 1, len(runs)))
df={}

scale_factors={'run':[], 'scale_factor':[]}
fig_height=10
fig_width=20

%matplotlib inline

nwide=3; nhigh=2
fig, (row1,row2) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
#fig.suptitle('Rows are')

for i,r in enumerate(runs):
    filename="data/"+date+'_'+test+"_"+r+"_metadata.csv"
#     print(filename)
    df[r] = pd.read_csv(filename, index_col=0)
    df[r].index=pd.to_datetime(df[r].index)
    # drop duplicates, not sure why there are double logs..
    df[r].drop_duplicates(subset=['elevationCalculatedAngle1','message','pressure'])

    if r =='initial':
        starting_pressure = df[r].pressure[0]

    scale_factor = 1-(df[r].pressure[0]-starting_pressure)/starting_pressure
    scale_factors['run'].append(r)
    scale_factors['scale_factor'].append(scale_factor)

#     print(f'ratio is {scale_factor:0.3f} for {r}')
    
    xvals = (df[r].m1_x_pos) # um
    yvals = df[r].elevationCalculatedAngle1
    sym=next(marker)
    label = f'{r} {scale_factor:0.3f}'
    color=colors[i]
    row1[0].plot(xvals, yvals, sym, color=color, label=label)
    row1[0].set_ylabel('Elevation [deg]')
    row1[0].set_xlabel('M1 X-position [mm]')
    
    xvals = (df[r].m1_y_pos) # mm
    row1[1].plot(xvals, yvals, sym, color=color, label=label)
    row1[1].set_ylabel('Elevation [deg]')
    row1[1].set_xlabel('M1 Y-position [mm]')

    xvals = (df[r].m1_piston) # mm
    row1[2].plot(xvals, yvals, sym, color=color, label=label)
    row1[2].set_ylabel('Elevation [deg]')
    row1[2].set_xlabel('M1 Z-position [mm]')

    row1[2].legend(bbox_to_anchor=(1.05, 1), loc='upper right', borderaxespad=0.)
    
    xvals = (df[r].m1_tip) # um   - Tip should mean azimuth motion
    row2[0].plot(xvals, yvals, sym, color=color, label=label)
    row2[0].set_xlabel('M1 tip [arcsec]')
    row2[0].legend(bbox_to_anchor=(1.05, 1), loc='upper right', borderaxespad=0.)
    
    xvals = (df[r].m1_tilt) # um
    row2[1].plot(xvals, yvals, sym, color=color, label=label)
    row2[1].set_xlabel('M1 tilt [arcsec]')

    xvals = (df[r].pressure) # um   - Tip should mean azimuth motion
    row2[2].plot(xvals, yvals, sym, color=color, label=label)
    row2[2].set_xlabel('Pressure [Pa]')

In [ ]:
for i in range(len(scale_factors['run'])):
    print(f'{scale_factors["run"][i]} {scale_factors["scale_factor"][i]:0.3f}')

In [ ]:
# For the fit we want to choose pressures from each curve, essentially assigning an elevation range for each run
# So first assign a range for each run
ranges={}
ranges['test15']=np.array((0,0)) # don't take data from this one
ranges['test14']=np.array((90,80))
ranges['test13']=np.array((80,76))
ranges['test12']=np.array((75,70))
ranges['test11']=np.array((70,65))
ranges['initial']=np.array((0,0)) # don't take data from this one
ranges['test3b']=np.array((64,58))
ranges['test4b']=np.array((57,49))
ranges['test5b']=np.array((48,45))
ranges['test6']=np.array((44,41))
ranges['test7']=np.array((40,39))
ranges['test8']=np.array((38,35))
ranges['test9']=np.array((34,33))
ranges['test10']=np.array((31,25))

In [ ]:
combined_data={'elevationCalculatedAngle1':np.zeros(0), 'pressure': np.zeros(0), 'm1_tip': np.zeros(0),
               'm1_tilt': np.zeros(0), 'm1_x_pos': np.zeros(0), 'm1_y_pos': np.zeros(0), 'm1_piston': np.zeros(0) }
elevation_range=np.arange(25,83,1)
for i,r in enumerate(runs):
    # skip the runs that we don't take data from 
    filename="data/"+date+'_'+test+"_"+r+"_metadata.csv"
    #print(filename)
    df[r] = pd.read_csv(filename, index_col=0)
    df[r].index=pd.to_datetime(df[r].index)
    # drop duplicates, not sure why there are double logs..
    df[r]=df[r].drop_duplicates(subset=['elevationCalculatedAngle1'])
    # loop over keys in combined data
    for key in combined_data.keys():
        # First need to sort the data by elevation
        args=np.argsort(df[r]['elevationCalculatedAngle1'].values)
        # first verify things are monotonically increasing
        xvals=df[r]['elevationCalculatedAngle1'].values[args]
        if ~np.all(np.diff(xvals) > 0):
            raise IOError(f'Values not monotonically increasing. Values are {xvals}')
        # Interpolate the data    
        int_data = np.interp(elevation_range, xvals,df[r][key].values[args])
        # find the data that corresponds to the range for that run
        ind=np.where(np.logical_and(elevation_range>=ranges[r][1], elevation_range<=ranges[r][0]))
        combined_data[key]=np.append(combined_data[key], int_data[ind],axis=0)
    #print(ind)
    #ranges[r]=np.array()

In [ ]:
# combined_data={'elevationCalculatedAngle1':np.zeros(0), 'pressure': np.zeros(0), 'm1_tip': np.zeros(0),
#                'm1_tilt': np.zeros(0), 'm1_x_pos': np.zeros(0), 'm1_y_pos': np.zeros(0), 'm1_piston': np.zeros(0) }
# elevation_range=np.arange(25,83,1)
# r='initial'
# filename=date+'_'+test+"_"+r+"_metadata.csv"
# df[r] = pd.read_csv(filename, index_col=0)
# df[r].index=pd.to_datetime(df[r].index)
# # drop duplicates, not sure why there are double logs..
# df[r]=df[r].drop_duplicates(subset=['elevationCalculatedAngle1'])
# key='m1_tip'
# # for key in combined_data.keys():
# # First need to sort the data by elevation
# args=np.argsort(df[r]['elevationCalculatedAngle1'].values)
# # Interpolate the data
# # first verify things are monotonically increasing
# xvals=df[r]['elevationCalculatedAngle1'].values[args]
# if ~np.all(np.diff(xvals) > 0):
#     raise IOError(f'Values not monotonically increasing. Values are {xvals}')
# int_data = np.interp(elevation_range, xvals,df[r][key].values[args])
# # find the data that corresponds to the range for that run
# ind=np.where(np.logical_and(elevation_range>=ranges[r][1], elevation_range<=ranges[r][0]))
# combined_data[key]=np.append(combined_data[key], int_data[ind],axis=0)

In [ ]:
fig_height=5
fig_width=10
nwide=2; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
#fig.suptitle('Rows are')

# Sort by elevation
args=np.argsort(combined_data['elevationCalculatedAngle1'])

xvals = (combined_data['m1_x_pos'][args]) # um
yvals = combined_data['elevationCalculatedAngle1'][args]
row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals = (combined_data['m1_tip'][args]) # um
row1[1].plot(xvals, yvals, 'o-')
row1[1].set_xlabel('M1 tip [arcsec]')

xvals = (combined_data['m1_y_pos'][args]) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')

xvals = (combined_data['m1_tilt'][args]) # um
row2[1].plot(xvals, yvals, 'o-')
row2[1].set_xlabel('M1 tilt [arcsec]')

xvals = (combined_data['m1_piston'][args]) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals = (combined_data['pressure'][args]) # mm
row3[1].plot(xvals, yvals, 'o-')
row3[1].set_ylabel('Elevation [deg]')
row3[1].set_xlabel('Pressure [Pa]')

In [ ]:
# Now do a fit to the pressure curve
# fit values from are
# m1:
#   -     -16218.270
#   -    +139909.826
#   -      -5150.417

In [ ]:
el=combined_data['elevationCalculatedAngle1'][args]
pressure=combined_data['pressure'][args]
fit_z = np.polyfit(np.cos(np.radians(90. - el)), pressure, 3)
poly_z = np.poly1d(fit_z)

fig_height=7
fig_width=10
nwide=1; nhigh=1
fig, ax = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))

ax.plot(el, pressure, '.')
ax.set_xlabel('Elevation [deg]')
ax.set_ylabel('Pressure [Pa]')
pressure_fit=poly_z(np.cos(np.radians(90. - el)))
ax.plot(el, pressure_fit,label='new_fit', color='red')
# plt.plot(el, rev_poly_z(np.cos(np.radians(90. - el))))

# plot the old fit
old_poly_z = np.poly1d([-16218.270, 139909.826, -5150.417])
old_pressure_fit=old_poly_z(np.cos(np.radians(90. - el)))
ax.plot(el, old_pressure_fit,'-', label='hex_m1_hex_20210707_v3.yaml', color='black')
ax.legend()

ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(el, pressure_fit*0.000145038,'.')
ax2.set_ylabel("Pressure [PSI]")

In [ ]:
fit_z

In [ ]:
# Compare against the others just to see
import itertools
import matplotlib.cm as cm
from matplotlib import colors
runs = ['initial','test3b','test4b','test5b','test6','test7','test8','test9','test10','test11','test12','test13','test14','test15']
#sym= ['.-','x-','triangle_up-']

marker = itertools.cycle((',-', '+--', 'x-', 'v-', '*-')) 
#colors=['red','blue','green']
colors = cm.rainbow(np.linspace(0, 1, len(runs)))
df={}

scale_factors={'run':[], 'scale_factor':[]}
fig_height=10
fig_width=20

%matplotlib inline

nwide=1; nhigh=1
fig, (row1) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
#fig, ax1 = plt.subplots()
#fig.suptitle('Rows are')

for i,r in enumerate(runs):
    filename=date+'_'+test+"_"+r+"_metadata.csv"
#     print(filename)
    df[r] = pd.read_csv(filename, index_col=0)
    df[r].index=pd.to_datetime(df[r].index)
    # drop duplicates, not sure why there are double logs..
    df[r].drop_duplicates(subset=['elevationCalculatedAngle1'])

    if r =='initial':
        starting_pressure = df[r].pressure[0]

    scale_factor = 1-(df[r].pressure[0]-starting_pressure)/starting_pressure
    scale_factors['run'].append(r)
    scale_factors['scale_factor'].append(scale_factor)
    
    yvals = df[r].elevationCalculatedAngle1
    sym=next(marker)
    label = f'{r} {scale_factor:0.3f}'
    color=colors[i]
#     print(f'ratio is {scale_factor:0.3f} for {r}')
 
    xvals = (df[r].pressure) # um   - Tip should mean azimuth motion
    row1.plot(xvals, yvals, sym, color=color, label=label)
    row1.set_xlabel('Pressure [Pa]')
    
    row1.legend(bbox_to_anchor=(1.05, 1), loc='upper right', borderaxespad=0.)
    
row1.plot(pressure_fit, el, label='new_fit', color='black')

In [ ]:
# The pressure transducer has a 0.1% error and comes in various ranges.
# We need to find out how much error we can tolerate in M1 position.

# plot the hexapod LUT 
hex_z = np.poly1d([-0.217, -1.045])
hex_z_data=hex_z(np.cos(np.radians(90. - el)))
fig,ax = plt.subplots()
ax.plot(el, hex_z_data,'-', label='m1_hex_20210817_v3.yaml', color='black')
ax.set_ylabel('Hexapod Z-position [mm]')
ax.set_xlabel('Elevation [deg]')
ax.legend()

In [ ]:
#so over the range, the hexapod moves 
dz=np.abs(np.max(hex_z_data)-np.min(hex_z_data))
delev=np.abs(np.max(el)-np.min(el))
print(f'variation is roughly {dz/delev:0.4f} [mm/deg]')

In [ ]:
# So 0.1% is X PSI which is X degress of M1 motion which is Y mm of hexapod motion, which is 

In [ ]:
# For a transducer
err_psi=(60-2)*0.001 # range is from 2-60 PSI, error is 0.1%
err_pa=err_psi/0.000145038
print(f'error is pascals is {err_pa:0.1f}')

In [ ]:
ind=np.where(np.logical_and(el>=28, el<=40))
fit_z_limited_range = np.polyfit(el[ind], pressure[ind], 1)
poly_z_limited_range = np.poly1d(fit_z_limited_range)
print(f'Fitting only this region says the pressure changes {fit_z_limited_range[1]:0.1f} [Pa/deg]')

In [ ]:
fig,ax = plt.subplots()

ax.plot(el, pressure, '.')
ax.set_xlabel('Elevation [deg]')
ax.set_ylabel('Pressure [Pa]')
pressure_fit=poly_z_limited_range(el)
ax.plot(el, pressure_fit,label='new_fit', color='red')
# ax.legend()

# ax2=ax.twinx()
# # make a plot with different y-axis using second axis object
# ax2.plot(el, pressure_fit*0.000145038,'.')
# ax2.set_ylabel("Pressure [PSI]")

In [ ]:
fit_z_limited_range

In [ ]:
# the fit from 28-40 in elevation gives the following about of Pa/degrees
err_motion_equivalent=err_pa/poly_z_limited_range[1]
print(f'the transducer pressure error corresponds to standard motion of {err_motion_equivalent:0.3f} [deg]')

In [ ]:
ind=np.where(np.logical_and(el>=28, el<=40))
fit_hex_z_limited_range = np.polyfit(el[ind], hex_z_data[ind], 1)
poly_hex_z_limited_range = np.poly1d(fit_hex_z_limited_range)
print(f'Fitting only this region says the hexapod changes {poly_hex_z_limited_range[1]:0.5f} [mm/deg]')

In [ ]:
hex_z_motion_err = poly_hex_z_limited_range[1]*err_motion_equivalent*1e3 # [um]
# Using a 4nm/um sensitivity (from the model)
print(f'The transducer error corresponds to {hex_z_motion_err:0.2f} [um], which is ~{4*hex_z_motion_err:0.2f}[nm]')

In [ ]:
# The pressure transducer has a 0.1% error and comes in various ranges.
# We need to find out how much error we can tolerate in M1 position.

# plot the hexapod LUT 
hex_z = np.poly1d([-0.217, -1.045])
hex_z_data=hex_z(np.cos(np.radians(90. - el)))
fig,ax = plt.subplots()
ax.plot(el, hex_z_data,'-', label='m1_hex_20210817_v3.yaml', color='black')
ax.set_ylabel('Hexapod Z-position [mm]')
ax.set_xlabel('Elevation [deg]')
ax.plot(el,poly_hex_z_limited_range(el),label='linear fit to restricted range')
ax.legend()